In [1]:
import os
import pandas as pd
import numpy as np
import data_wrangle
import json
from importlib import reload
import preprocess
import sqlite3
import analysis
import prettytable as pt


#From PCA
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Leave one out
from sklearn.model_selection import LeaveOneOut

import scipy.signal as ss
import pybaselines.spline as py

import pandas as pd
import numpy as np
from math import ceil
import statistics

# import plotly.io as pio
# pio.renderers.default = "browser"
# pio.renderers.default = "png"



# Using the DataTable class

---

+ Create a new table object: </br>
`db=data_wrange.DataTable('path/to/sql.db', 'table_name')` </br>
If you do not enter a `table_name`, a list of tables available in the database will be displayed.

+ Grab Raman Intensity values  
`ys=db.intens()`  
Each row will be an individual spectra, where index = sampleID.  
Each column is a Raman Shift.

+ Grab Raman Shifts  
`x=db.raman_shifts()`

+ Generate labels for the sample rows
    1. Use default names or set your own  
        `db.name_cols=['col_name(s)']`
    2. If more than 1 column is give, the `db.name_join` attribute can be updated.
    3. Generate labels
        + DataFrame Index  
        This is the method i prefer, but it may need more testing
        `namedic=db.name_dic()`  
        `df=data_wrangle.map_index(ys,namedic)`
        + List based on index
        `namelist=db.names(ys)`  
        The input dataframe needs the index to be the sampleID.  
        The `intens()` function sets index as sampleID automatically.

In [23]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/dopamine.db', 'da_dopac')
x=db.raman_shifts()
ys=db.intens()

namedic=db.name_dic(cols=['DA'])
symdic=db.name_dic(cols=['DOPAC'])
sampledic=db.name_dic(cols=['DA','DOPAC']) #Use concat sql method?



{(282, 283, 285, 286, 287, 289, 290, 292, 293, 295, 296, 298, 299, 301, 302, 303, 305, 306, 308, 309, 311, 312, 314, 315, 317, 318, 319, 321, 322, 324, 325, 327, 328, 330, 331, 332, 334, 335, 337, 338, 340, 341, 343, 344, 345, 347, 348, 350, 351, 353, 354, 356, 357, 358, 360, 361, 363, 364, 366, 367, 368, 370, 371, 373, 374, 376, 377, 379, 380, 381, 383, 384, 386, 387, 389, 390, 391, 393, 394, 396, 397, 399, 400, 401, 403, 404, 406, 407, 409, 410, 411, 413, 414, 416, 417, 418, 420, 421, 423, 424, 426, 427, 428, 430, 431, 433, 434, 436, 437, 438, 440, 441, 443, 444, 445, 447, 448, 450, 451, 453, 454, 455, 457, 458, 460, 461, 462, 464, 465, 467, 468, 469, 471, 472, 474, 475, 476, 478, 479, 481, 482, 484, 485, 486, 488, 489, 491, 492, 493, 495, 496, 498, 499, 500, 502, 503, 505, 506, 507, 509, 510, 512, 513, 514, 516, 517, 519, 520, 521, 523, 524, 526, 527, 528, 530, 531, 532, 534, 535, 537, 538, 539, 541, 542, 544, 545, 546, 548, 549, 551, 552, 553, 555, 556, 557, 559, 560, 562, 563, 564

In [24]:
#Truncate and check attribute compatability (would be nice error message)

df=ys.iloc[:,140:-700]
xx=x[140:-700]

print(f'lengths || df: {df.shape}, xx: {len(xx)}')

lengths || df: (500, 453), xx: 453


In [28]:
reload(analysis)
var=analysis.multivar(df,xx,sampledic)
                    #   symbol=symdic)

var.pca_3d()

dictionary for color labels exists


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression as lr

In [75]:
# Loop for MSE calcuations
# https://www.statology.org/leave-one-out-cross-validation-in-python/


def get_split(test,whole):
    rest=[val for val in whole if val != test]
    return test, rest
    
input_df=data

mse_dic={}

mse_list=[]
for i in range(len(input_df)): #for each row in the pca_scores dataframe
    test=input_df.iloc[i:] #grab the row data
    rest_index=[val for val in list(range(len(input_df))) if val != i]
    rest=input_df.iloc[rest_index] #grab the remaining data

    cv=LeaveOneOut()
    model=lr()
    # print(test.T.shape)

    scores=cross_val_score(model, test.T, rest.T, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    score=np.mean(np.absolute(scores))
    mse_list.append(score)

    #Make a dictionary so MSE values by sample name can be seen
    name=input_df.iloc[i].name #get the sample name from its index
    if name in mse_dic: #check if the sample name already exists in the dictionary
        mse_dic[name].append(round(score,4)) #if the sample is there, append the new value to the list
    else: #if the sample name is not in the dictionary, add it (create a new key)
        mse_dic[name]=[round(score,4)]

In [76]:
for key in mse_dic:
    print(key)
    lst=mse_dic[key]
    print(lst)
    print(f'Average: {round(statistics.mean(lst),4)}')
    print(f'STD: {round(statistics.stdev(lst),4)}')
    print()

7 uM, NaN
[0.1911, 0.1911, 0.191, 0.191, 0.1909, 0.194, 0.1941, 0.1941, 0.1941, 0.1941, 0.1941, 0.1941, 0.1941, 0.1941, 0.1942, 0.1941, 0.1942, 0.1942, 0.1942, 0.1943, 0.1943, 0.1941, 0.1944, 0.1944, 0.1945]
Average: 0.1936
STD: 0.0013

7 uM, 3 uM
[0.191, 0.191, 0.191, 0.191, 0.191, 0.1911, 0.1911, 0.1911, 0.1912, 0.1912, 0.1912, 0.1912, 0.1913, 0.1913, 0.1913, 0.1913, 0.1913, 0.1915, 0.1915, 0.1916, 0.1916, 0.1917, 0.1918, 0.1918, 0.1919]
Average: 0.1913
STD: 0.0003

3 uM, 7 uM
[0.1919, 0.1918, 0.1917, 0.1917, 0.1916, 0.1915, 0.1914, 0.1914, 0.1914, 0.1915, 0.192, 0.1921, 0.1922, 0.1922, 0.1923, 0.1924, 0.1926, 0.1927, 0.1929, 0.193, 0.193, 0.1932, 0.1934, 0.1935, 0.1937]
Average: 0.1923
STD: 0.0007

10 uM, NaN
[0.1946, 0.1946, 0.1948, 0.1949, 0.195, 0.1951, 0.1952, 0.1954, 0.1955, 0.1957, 0.196, 0.196, 0.1961, 0.1962, 0.1964, 0.1966, 0.1967, 0.1968, 0.197, 0.1971, 0.1974, 0.1975, 0.1976, 0.1978, 0.198]
Average: 0.1962
STD: 0.0011

3 uM, NaN
[0.1984, 0.1984, 0.1984, 0.1985, 0.1985, 0.

In [77]:
#average MSE
print(sum(mse_list)/(len(mse_list)))

#view RMSE
np.sqrt(np.mean(np.absolute(scores)))

0.24237121678270782


0.4795329078132311

<h1> PCA

In [104]:
reload(analysis)
var=analysis.multivar(ys,x,ys.index,ys.index,ncomp=10)
data,b,c=var.pca_run()

In [20]:
#Truncate and check attribute compatability (would be nice error message)

df=ys.iloc[:,140:-700]
xx=x[140:-700]

print(f'lengths || df: {df.shape}, xx: {len(xx)}')

lengths || df: (500, 453), xx: 453


<h1> Data Work

<h2>Get Data From DB

<h1> Analysis

In [7]:
var=multivar(ys.T,x,group,group)

# figs=var.pca_figs('PC 3', 'PC 3')
# f=figs[1]
# f.update_layout(font=dict(family='Ariel', size=30))
# f.write_image('/mnt/c/Users/16162/Downloads/%s.svg'%'sheep2_pc3', width=1600, height=800)

# f=var.pca_3d(color=lst)#,color_dict=colorkey)
# f.update_layout(font=dict(family='Arial', size=18),
#                 margin=dict(b=50))
# # f.write_image('/mnt/c/Users/16162/Downloads/%s.svg'%'sheep2_pca', width=1600, height=1200)
# f.show()

In [ ]:
# pio.renderers.default = "svg"
var=multivar(ys.T,x,lst,lst)

# figs=var.pca_figs('PC 3', 'PC 3')
# f=figs[1]
# f.update_layout(font=dict(family='Ariel', size=30))
# f.write_image('/mnt/c/Users/16162/Downloads/%s.svg'%'sheep2_pc3', width=1600, height=800)

f=var.pca_3d(color=lst)#,color_dict=colorkey)
# f.update_layout(font=dict(family='Arial', size=18),
#                 margin=dict(b=50))
# # f.write_image('/mnt/c/Users/16162/Downloads/%s.svg'%'sheep2_pca', width=1600, height=1200)
f.show()

In [ ]:
var=multivar(ys.T,x,names,hover,symbol=lst1, hover=hover)

<h1>Update Database Find and Replace

In [37]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','sheep')
# db.condition='WHERE sheep IS 2'
db.data_info('nanotag') #Add 'condition' to find_replace

db.find_replace('2023-04-13', '1','nanotag')

Number of samples selected: 280


[+----------------------+
 |       nanotag        |
 +----------+-----------+
 |  Value   | Frequency |
 +----------+-----------+
 |   0min   |     6     |
 |  10min   |     12    |
 |  15min   |     12    |
 |  20min   |     27    |
 |  25min   |     12    |
 |  30min   |     12    |
 |  35min   |     12    |
 |  40min   |     12    |
 |   5min   |     12    |
 | NBA 0min |     6     |
 |   both   |     87    |
 |    no    |     70    |
 +----------+-----------+]